<a href="https://colab.research.google.com/github/saccordi/InsightPlaces/blob/master/InsightPlaces_transformando_os_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bem-vindo(a) ao meu notebook!**
Com o objetivo de ajudar a imobiliária *InsightPlaces* no seu problema de mercado, é necessário começar com o processo de transformação de dados.

Ao longo desse noteebook, serão realizados algumas modificações nos dados brutos como remoção e seleção de informações, alteração de tipo de dados e junçõa de informações. Por fim, o resultado das transformacões será salvo no formato parquet que possui uma performance melhor em relação ao cvs.

In [ ]:
# instalando o PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=fcf487b30623df3e0d959799b2b20d13e91f5c26722d849436246e2793428b7a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# iniciando uma sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Transformacao com Spark") \
    .getOrCreate()

spark

## Obtenção dos dados
Os dados foram disponibilizados pela própria *InsightPlaces* e estão no formato JSON que contem uma estrutura complexa com arrays e structs.

In [ ]:
# obtendo os dados dos imóveis
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip' && unzip semana-1.zip -d dados/

--2023-11-22 21:55:55--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.42.217, 52.217.230.193, 52.216.79.44, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.42.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip’

semana-1.zip        100%[===================>]  18.10M  21.3MB/s    in 0.8s    

2023-11-22 21:55:57 (21.3 MB/s) - ‘semana-1.zip’ saved [18975214/18975214]

Archive:  semana-1.zip
  inflating: dados/dataset_bruto.json  


In [ ]:
# realizando a leitura da base de dados
dados = spark.read.json(
    'dados/dataset_bruto.json'
)

In [ ]:
# analisando a estrutura da base de dados
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [ ]:
# analisando o número de linhas e colunas do conjunto de dados
print("Número de linhas do conjunto de dados:", dados.count())

Número de linhas do conjunto de dados: 89083


## Seleção dos dados
Para nossa análise, apenas as informações do campo `anuncio` serão relevantes. Por isso, vamos focar em analisar as colunas desse campo.

### Dicionário de dados


| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

In [ ]:
# selecionando os dados de interesse para o projeto
dados\
    .select(
        'anuncio.*'
    )\
    .show(truncate=False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+

In [ ]:
# atribuindo os dados de interesse a uma variável
anuncio=dados.select('anuncio.*')

## Filtrando a base de dados
A *InsightPlaces* solicitou que fossem feitos alguns filtros nas colunas `tipo_uso`, `tipo_unidade` e `tipo_anuncio` da base de dados.

### Avaliando a coluna `tipo_uso`

In [ ]:
# criando um tabela de frequências segundo o tipo de uso dos imóveis
anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



Como há apenas 4542 imóveis comerciais, a *InsightPlaces* optou por deixá-los de fora do projeto.

In [ ]:
# filtrando somente imóveis residenciais
anuncio=anuncio.filter('tipo_uso=="Residencial"')

In [ ]:
# visualizando novamente a tabela de frequências segundo o tipo de uso dos imóveis
anuncio\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
+-----------+-----+



### Avaliando a coluna `tipo_unidade`

In [ ]:
# criando um tabela de frequências segundo o tipo de unidade dos imóveis
anuncio\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



O Dataset apresenta 3 tipos de unidades (Outros, Apartamento e Casa). A *InsightPlaces* optou por trabalhar somente com o tipo Apartamento.

In [ ]:
# filtrando imóveis somente do tipo apartamento
anuncio=anuncio.filter('tipo_unidade=="Apartamento"')

In [ ]:
# visualizando novamente a tabela de frequências segundo o tipo de unidade
anuncio\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66797|
+------------+-----+



## Avaliando as informações de configuração dos imóveis

As  colunas `quartos`, `suites`, `banheiros`, `vaga`, `area_total`, `area_util` definem as configurações dos imóveis. É importante reforçar que elas estão em formato de listas. Isso não é por acaso.

A *InsightPlaces* comercializa dois tipos de imóveis, usados e novos (lançamentos). Os imóveis lançamentos são aqueles que estão em construção e pertencem a um determinado empreendimento. Esses empreendimentos podem conter mais de um tipo de configuração para os imóveis (exemplo: imóveis de 2 a 4 quartos).

Sendo assim, os imóveis usados são preenchidos apenas com um valor (exemplo: imóvel de 2 quartos -> [2]) e os lançamentos podem ser preenchidos com apenas um valor (quando o empreendimento tem imóveis com a mesma configuração) ou com dois valores (quando o empreendimento apresenta imóveis com mais configurações. Exemplos: imóveis de 2 a 4 quartos -> [2, 4]).
Para melhor visualização, será feito um filtro para mostrar apenas imóveis do tipo lançamento (seram selecionadas apenas as colunas `quartos`, `suites`, `banheiros`, `vaga`, `area_total`, `area_util`.

In [ ]:
# visualizando  imóveis do tipo lançamento
# somente colunas quartos, suites, banheiros, vaga, area_total, area_util
anuncio\
    .select(
        'quartos',
        'suites',
        'banheiros',
        'vaga', 'area_total',
        'area_util'
        )\
    .filter('tipo_anuncio=="Lançamento"')\
    .show()

+-------+------+---------+------+----------+---------+
|quartos|suites|banheiros|  vaga|area_total|area_util|
+-------+------+---------+------+----------+---------+
| [1, 2]|   [0]|      [0]|   [0]|        []| [41, 51]|
| [1, 2]|   [0]|      [0]|   [0]|        []|     [41]|
|    [0]|   [0]|      [1]|   [0]|        []| [21, 22]|
|    [0]|   [0]|      [0]|   [0]|        []|[20, 159]|
|    [0]|   [0]|      [0]|   [0]|        []|[22, 150]|
|    [0]|   [0]|      [0]|   [1]|        []| [20, 51]|
|    [0]|   [0]|      [0]|   [0]|        []|[27, 202]|
| [1, 2]|   [1]|      [2]|   [1]|        []|[39, 112]|
|    [2]|   [0]|      [1]|   [1]|        []| [41, 45]|
| [2, 3]|   [0]|      [1]|   [1]|        []| [43, 52]|
|    [2]|   [0]|      [1]|   [1]|        []| [41, 45]|
| [1, 2]|[1, 2]|   [1, 2]|[1, 2]|        []| [30, 66]|
|    [2]|   [0]|      [1]|   [1]|        []| [43, 48]|
| [1, 2]|   [0]|      [1]|   [1]|        []| [35, 40]|
| [1, 2]|   [0]|      [1]|[0, 1]|        []| [32, 43]|
| [1, 2]|[

Após essas analises, a *InsightPlaces* optou por dar proseguimento somente com imóveis usados. Além disso, será necessário transformar os dados em formato de lista para o formato numérico. Essa transformação vai possibilitar a utilização dos dados por algoritmos de machine learning.

In [ ]:
# visualizando o total de imóveis lançamento e usados
anuncio\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



In [ ]:
# filtrando imóveis usados
anuncio=anuncio.filter('tipo_anuncio=="Usado"')

In [ ]:
# visualizando novamente a tabela de frequências segundo o tipo de anuncio
anuncio\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



## Transformando as configurações dos imóveis de listas para inteiros

In [ ]:
# importando o módulo necessário para transformar os dados
from pyspark.sql import functions as f

In [ ]:
# confirmando que tem apenas listas com apenas um elemento
anuncio\
  .select(f.size(f.col('quartos')).alias('quartos'))\
  .groupBy('quartos')\
  .count()\
  .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+



In [ ]:
# criando um lista com as colunas de interesse
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [ ]:
# verificando os registros das outras colunas
for col in caracteristicas:
    anuncio.select(f.size(f.col(col)).alias(col)).groupBy(col).count().show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



Nota-se que as colunas `suites`, `vaga` e `area_total` possuem listas vazias em alguns registros. No caso das colunas `suites` e `vaga`, é possível considerar que de fato esses imóveis não possuem essas configurações e pode-se imputar o valor zero. Já no caso da `area_total` não é possível fazer essa inferência, pois um imóvel precisa ter uma área. Ou seja, o vazio é resultado de outros fatores.

In [ ]:
# substituindo as listas por valores
anuncio\
  .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])\
  .show(truncate = False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

In [ ]:
# aplicando as mudanças ao Dataframe
anuncio=anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

In [ ]:
# mostrando o Dataset atualizado
anuncio.show(truncate = False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

## Tratando as informações sobre localização
Como a *InsightPlaces* opera somente no estado do Rio de Janeiro, a imobiliária solicitou que fossem extraídos apenas os dados sobre bairro e zona. Sendo assim, será necessário transformar a coluna `endereço`.

In [ ]:
# visualizando a coluna endereço
anuncio\
  .select('endereco.*')\
  .show()

+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|
+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|R

In [ ]:
# visualizando apenas o bairro e zona da coluna endereço
anuncio\
  .select('endereco.bairro', 'endereco.zona')\
  .show()

+--------------------+----------+
|              bairro|      zona|
+--------------------+----------+
|           Paciência|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|  Pedra de Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|        Campo Grande|Zona Oeste|
|          Santa Cruz|Zona Oeste|
|           Guaratiba|Zona Oeste|
|Recreio dos Bande...|Zona Oeste|
|              Cosmos|Zona Oeste|
|            Realengo|Zona Oeste|
|              Cosmos|Zona Oeste|
|     Todos os Santos|Zona Norte|
+--------------------+----------+
only showing top 20 rows



In [ ]:
# visualizando o Dataset com apenas as colunas bairro e zona
anuncio\
  .select('*',  'endereco.bairro', 'endereco.zona')\
  .drop('endereco')\
  .show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  NULL|       Usado| Apartamento|Residencial|   1|[{245, NULL, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  NULL|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|[P

In [ ]:
# aplicando as mudanças ao Dataset
anuncio=anuncio\
  .select('*',  'endereco.bairro', 'endereco.zona')\
  .drop('endereco')

## Tratando a coluna valores

Essa é a informação mais importante papa a *InsightPlaces*. Percebe-se que a coluna `valores` é do tipo array, ou seja, um lista que contém elementos. Esses elementos são dicionários com informações sobre `condominio`, `iptu`, `tipo` e `valor`.

O campo `tipo` pode assumir duas formas: venda ou aluguel.

O campo `valor` é o preço do imóvel, seja o valor de venda ou aluguel.

Os campos `condominio`, `iptu` são as taxas extras do imóvel.

Como se trata de um projeto sobre a venda, a imobiliaria se interessa apenas por imóveis dessa categoria.

In [ ]:
# visualizando a estrutura dos dados
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



In [ ]:
# visualizando a coluna valores
anuncio\
  .select('valores')\
  .show()

+--------------------+
|             valores|
+--------------------+
|[{245, NULL, Vend...|
|[{0, 0, Venda, 15...|
|[{0, 0, Venda, 20...|
|[{285, NULL, Vend...|
|[{245, NULL, Vend...|
|[{285, NULL, Vend...|
|[{250, NULL, Vend...|
|[{245, NULL, Vend...|
|[{245, NULL, Vend...|
|[{240, NULL, Vend...|
|[{0, 0, Venda, 15...|
|[{240, 0, Venda, ...|
|[{245, NULL, Vend...|
|[{290, NULL, Vend...|
|[{285, NULL, Vend...|
|[{0, 0, Venda, 30...|
|[{NULL, NULL, Ven...|
|[{280, 0, Venda, ...|
|[{280, NULL, Vend...|
|[{0, 0, Venda, 26...|
+--------------------+
only showing top 20 rows



In [ ]:
# verificando quantos imóveis a venda têm no conjunto de dados
anuncio\
  .select(f.explode('valores').alias('valores'))\
  .select('valores.*')\
  .filter('valores.tipo=="Venda"')\
  .count()

66562

In [ ]:
# transformando a coluna valores
anuncio\
  .select('id', f.explode('valores').alias('valores'))\
  .select('id', 'valores.*')\
  .show(100, truncate=False)

+------------------------------------+----------+----+-------+-----+
|id                                  |condominio|iptu|tipo   |valor|
+------------------------------------+----------+----+-------+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |NULL|Venda  |15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda  |15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda  |20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |NULL|Venda  |20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |NULL|Venda  |15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |NULL|Venda  |20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |NULL|Venda  |15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |NULL|Venda  |15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |NULL|Venda  |15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |NULL|Venda  |17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda  |15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12

In [ ]:
# transformando a coluna valores e filtrando apenas os imóveis a venda
anuncio\
  .select('id', f.explode('valores').alias('valores'))\
  .select('id', 'valores.*')\
  .filter('valores.tipo=="Venda"')\
  .show(truncate=False)

+------------------------------------+----------+----+-----+-----+
|id                                  |condominio|iptu|tipo |valor|
+------------------------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |NULL|Venda|15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda|15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda|20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |NULL|Venda|20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |NULL|Venda|15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |NULL|Venda|20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |NULL|Venda|15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |NULL|Venda|15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |NULL|Venda|15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |NULL|Venda|17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda|15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12014|240       |0   |Venda|19

In [ ]:
# atribuindo as mudanças a uma variável
valores=anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')

In [ ]:
# contando os registros da união dos valores com outras infomações
anuncio.join(valores, 'id', how='inner').count()

66562

In [ ]:
# juntando as colunas de valores formatadas às outras informações
anuncio.join(valores, 'id', how='inner').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  NULL|       Usado| Apartamento|Residencial|   1|[{285, NULL, Vend...|            Realengo|  Zona Oeste|       285|NULL|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado|

In [ ]:
# removendo as informações de valores não desejadas
anuncio.join(valores, 'id', how='inner').drop('valores').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  NULL|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|NULL|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [ ]:
# aplicando as modificações ao Dataframe
anuncio=anuncio.join(valores, 'id', how='inner').drop('valores')

## Salvando o resultado da extração no formato parquet

In [ ]:
# salvando o Dataframe em parquet e sobrescrevendo o arquivo já existente
anuncio.write.parquet(
    path='dados_salvos/dataset_transformado_parquet',
    mode='overwrite'
)

In [ ]:
# realizando a leitura do arquivo parquet
anuncio_parquet = spark.read.parquet(
    'dados_salvos/dataset_transformado_parquet'
)

In [ ]:
# visualizando a estrutura do do arquivo parquet
anuncio_parquet.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [ ]:
# visualizando a quantidade de registros
anuncio_parquet.count()

66562

## Salvando no formato CSV

In [ ]:
# salvando os dados transformados
anuncio = anuncio.drop("caracteristicas")

In [ ]:
# sobreescrevendo o arquivo atual
anuncio.write.csv(
    path='dados_salvos/dataset_transformado.csv',
    mode='overwrite',
    header=True
)

In [ ]:
# realizando a leitura do arquivo csv
anuncio_csv = spark.read.csv(
    'dados_salvos/dataset_transformado.csv',
    header=True
)

In [ ]:
# visualizando a estrutura do do arquivo csv
anuncio_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: string (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: string (nullable = true)
 |-- quartos: string (nullable = true)
 |-- suites: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## Comparando o desempenho entre os arquivos parquet e csv

In [ ]:
# visualizando o desempendo do arquivo parquet
%%time
anuncio_parquet.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|       Tomás Coelho|   38|
|              Gávea|  597|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|      Bento Ribeiro|   33|
|              Rocha|   38|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|       Campo Grande|  639|
|        Jacarepaguá| 4437|
|               Leme|  234|
|           Botafogo| 3499|
|       Padre Miguel|   15|
|       Santo Cristo|  357|
|           Flamengo| 1378|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 9.37 ms, sys: 816 µs, total: 10.2 ms
Wall time: 454 ms


In [ ]:
# visualizando o desempendo do arquivo csv
%%time
anuncio_csv.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|       Tomás Coelho|   38|
|              Gávea|  597|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|      Bento Ribeiro|   33|
|              Rocha|   38|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|       Campo Grande|  639|
|        Jacarepaguá| 4437|
|               Leme|  234|
|           Botafogo| 3499|
|       Padre Miguel|   15|
|       Santo Cristo|  357|
|           Flamengo| 1378|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 15.1 ms, sys: 1.9 ms, total: 17 ms
Wall time: 1.3 s
